# Make nice bed files for Fly Meeting

In [1]:
import pandas as pd
import gffutils
from gffutils import pybedtools_integration
import pybedtools
from pybedtools.featurefuncs import gff2bed

### MOTIFS:

In [2]:
df = pd.read_table('../output/motif/tf_merge.bed', header=None, 
              names=['chrom', 'start','end','pval','motif_symbol','motif_FBgn'])

In [4]:
#Filter for RNAi TFs
TF_list = pd.read_table('../output/list_of_tfs.txt', header=None)
TF_list.columns=['TF']

merge_on_our_TFs = df.merge(TF_list, left_on='motif_FBgn', right_on='TF', how='inner')

In [6]:
bed = merge_on_our_TFs[['chrom', 'start','end','motif_FBgn', 'pval']]

In [7]:
bed.head()

,chrom,start,end,motif_FBgn,pval
0,chr2L,6782,6789,FBgn0283451,0.000036
1,chr2L,17019,17026,FBgn0283451,0.000036
2,chr2L,19846,19853,FBgn0283451,0.000036
3,chr2L,29403,29410,FBgn0283451,0.000036
4,chr2L,30047,30054,FBgn0283451,0.000036


In [8]:
bed.to_csv('../../datashare/tf_motifs.bed', sep='\t', index=False)

### CHIP:

In [9]:
tf = pd.read_table('../output/chip/ALL_TF_CHIP_filtered.bed', header=None, 
                   names=['chrom','start','end','srx','score','caller'])

In [ ]:
tf